In [ ]:
%%time
!pip3 install -U sagemaker
!pip3 install -U boto3

#then restart kernel

In [ ]:
%%time
import os
import json

import sagemaker
import boto3
from sagemaker.pytorch import PyTorch
from sagemaker import get_execution_role
import torch
from sagemaker.utils import unique_name_from_base,name_from_base

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = get_execution_role()
prefix = 'transsion-poc-zhuan'
output_path = f"s3://{bucket}/{prefix}"

print("torch.__version__:{}".format(torch.__version__))
print("boto3.__version__:{}".format(boto3.__version__))
print("sagemaker.__version__:{}".format(sagemaker.__version__))
print("bucket:{}".format(bucket))
print("role:{}".format(role))

In [ ]:
%%time
!rm -fr wenet
!git clone https://github.com/wenet-e2e/wenet.git
!cp wenet/examples/librispeech/s0/*.sh wenet
!cp -r wenet/examples/librispeech/s0/local wenet
!cp -r wenet/examples/librispeech/s0/conf wenet
!rm -fr wenet/examples

In [ ]:
%%time
#download data.list etc.
#change to your bucket & prefix that saving your data.list and other files.
# ****** if the flac file path is not correct, please replace it manually. ******
#!aws s3 cp --recursive s3://<bucket>/<prefix>/data wenet/data
#!aws s3 cp --recursive s3://<bucket>/<prefix>/exp wenet/exp
!aws s3 cp --recursive s3://sagemaker-us-east-1-432088571089/dataset-transsion-new/data wenet/data
!aws s3 cp --recursive s3://sagemaker-us-east-1-432088571089/dataset-transsion-new/exp wenet/exp


In [ ]:
#change run.sh
run_bak_sh='wenet/run_bak.sh'
if(not os.path.exists(run_bak_sh)):
    !cp wenet/run.sh wenet/run_bak.sh

with open(run_bak_sh, 'r') as file:
    # read a list of lines into data
    data = file.readlines()

    # now change the 2nd line, note that you have to add a newline
    data[8] = 'export CUDA_VISIBLE_DEVICES="0,1,2,3,4,5,6,7"\n' # change this according the number of GPUs
    data[9] = 'stage=4 # start from 0 if you need to start from data preparation\n'
    data[10] = 'stop_stage=4\n'
    data[14] = 'datadir=/opt/ml/input/data/training\n'
    data[130] = '  dist_backend="nccl"\n'
    data[135] = '  pip install -r requirements.txt\n'
    
    data[147] = '      --model_dir /opt/ml/model \\\n'

    # and write everything back
    with open('wenet/run.sh', 'w') as file:
        file.writelines( data )

In [ ]:
%%time
job_name = name_from_base(prefix)
use_spot_instances = False # spot or not
max_run = 360000 
max_wait = 7200 if use_spot_instances else None
checkpoint_s3_uri = (
    "s3://{}/{}/checkpoints/{}".format(bucket, prefix, job_name) if use_spot_instances else None
)

instance_type = "ml.p4d.24xlarge"#"ml.p3.16xlarge"

est = PyTorch(
    entry_point="run.sh",
    source_dir="./wenet",
    framework_version="1.12.1", #1.11.0success #1.10.2error
    py_version="py38",
    role=role,
    instance_count=1,
    instance_type=instance_type,
    volume_size=250,
    disable_profiler=True,
    debugger_hook_config=False,
    job_name=job_name,
    use_spot_instances=use_spot_instances,
    max_run=max_run,
    max_wait=max_wait,
    checkpoint_s3_uri=checkpoint_s3_uri,
)


In [ ]:
from sagemaker.inputs import TrainingInput
prefix_dataset = "dataset-transsion-new/export"
loc =f"s3://{bucket}/{prefix_dataset}"

training = TrainingInput(
    s3_data_type='S3Prefix', # Available Options: S3Prefix | ManifestFile | AugmentedManifestFile
    s3_data=loc,
    distribution='FullyReplicated', # Available Options: FullyReplicated | ShardedByS3Key 
    input_mode='FastFile'
)

In [ ]:
import numpy as np

# Declaring and Initializing the one Dimension Array
x = np.array([212,312,2,12,124,142,12])

# Printing the result
print(x[0,3])

In [ ]:
%%time
job_name = est.fit({"training":training})
#job_name = est.fit()

In [ ]:
model_data = est.model_data
print("Model artifact saved at:\n", model_data)